In [39]:
!pip install groq

In [40]:
!pip install python-dotenv

In [41]:
import json

def get_current_weather(location: str, unit: str):
	"""
	Get the current weather in a given location

	location (str): The city and state, e.g. Madrid, Barcelona
	unit (str): The unit. It can take two values; "celsius", "fahrenheit"
	"""
	if location == "Madrid":
		return json.dumps({"temperature": 25, "unit": unit})

	else:
		return json.dumps({"temperature": 58, "unit": unit})

In [42]:
get_current_weather("Madrid", "celsius")

'{"temperature": 25, "unit": "celsius"}'

In [43]:
import os
import re
from groq import Groq
from dotenv import load_dotenv

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""


In [44]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]
agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "What's the current temperature in Madrid, in Celsius?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}}
</tool_call>


In [45]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)
    
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [46]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Madrid', 'unit': 'celsius'}}

In [47]:
result = get_current_weather(**parsed_output["arguments"])
result

'{"temperature": 25, "unit": "celsius"}'

In [48]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [49]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current temperature in Madrid is 25°C.'

In [50]:
!pip install requests

In [51]:
!pip install colorama


In [53]:
import json
import requests

from src.agentic_patterns.tool_pattern.tool import tool
from src.agentic_patterns.tool_pattern.tool_agent import ToolAgent

def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. 
    Each story contains the title, URL, score, author, and time of submission. The data is fetched 
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    
    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Get the top story IDs
        top_story_ids = response.json()[:top_n]
        
        top_stories = []
        
        # For each story ID, fetch the story details
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors
            story_data = story_response.json()
            
            # Append the story title and URL (or other relevant info) to the list
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })
        
        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

In [54]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': 'News publishers limit Internet Archive access due to AI scraping concerns',
  'url': 'https://www.niemanlab.org/2026/01/news-publishers-limit-internet-archive-access-due-to-ai-scraping-concerns/'},
 {'title': 'uBlock filter list to hide all YouTube Shorts',
  'url': 'https://github.com/i5heu/ublock-hide-yt-shorts/'},
 {'title': "My smart sleep mask broadcasts users' brainwaves to an open MQTT broker",
  'url': 'https://aimilios.bearblog.dev/reverse-engineering-sleep-mask/'},
 {'title': 'IBM tripling entry-level jobs after finding the limits of AI adoption',
  'url': 'https://fortune.com/2026/02/13/tech-giant-ibm-tripling-gen-z-entry-level-hiring-according-to-chro-rewriting-jobs-ai-era/'},
 {'title': 'Ooh.directory: a place to find good blogs that interest you',
  'url': 'https://ooh.directory/'}]

In [55]:
hn_tool = tool(fetch_top_hacker_news_stories)

In [56]:
hn_tool.name

'fetch_top_hacker_news_stories'

In [57]:
json.loads(hn_tool.fn_signature)

{'name': 'fetch_top_hacker_news_stories',
 'description': '\nFetch the top stories from Hacker News.\n\nThis function retrieves the top `top_n` stories from Hacker News using the Hacker News API. \nEach story contains the title, URL, score, author, and time of submission. The data is fetched \nfrom the official Firebase Hacker News API, which returns story details in JSON format.\n\nArgs:\n    top_n (int): The number of top stories to retrieve.\n',
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

In [58]:
tool_agent = ToolAgent(tools=[hn_tool])

In [59]:
output = tool_agent.run(user_msg="Tell me your name")

KeyError: 'None'

In [60]:
print(output)

<tool_call>
{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}}
</tool_call>


In [61]:
output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now")


Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'name': 'fetch_top_hacker_news_stories', 'arguments': {'top_n': 5}, 'id': 1}

Tool result: 
[{"title": "News publishers limit Internet Archive access due to AI scraping concerns", "url": "https://www.niemanlab.org/2026/01/news-publishers-limit-internet-archive-access-due-to-ai-scraping-concerns/"}, {"title": "uBlock filter list to hide all YouTube Shorts", "url": "https://github.com/i5heu/ublock-hide-yt-shorts/"}, {"title": "My smart sleep mask broadcasts users' brainwaves to an open MQTT broker", "url": "https://aimilios.bearblog.dev/reverse-engineering-sleep-mask/"}, {"title": "IBM tripling entry-level jobs after finding the limits of AI adoption", "url": "https://fortune.com/2026/02/13/tech-giant-ibm-tripling-gen-z-entry-level-hiring-according-to-chro-rewriting-jobs-ai-era/"}, {"title": "Ooh.directory: a place to find good blogs that interest you", "url": "https://ooh.directory/"}]


In [62]:
print(output)

Based on the provided data, here are the top 5 Hacker News stories right now:

1. **News publishers limit Internet Archive access due to AI scraping concerns** - https://www.niemanlab.org/2026/01/news-publishers-limit-internet-archive-access-due-to-ai-scraping-concerns/
2. **uBlock filter list to hide all YouTube Shorts** - https://github.com/i5heu/ublock-hide-yt-shorts/
3. **My smart sleep mask broadcasts users' brainwaves to an open MQTT broker** - https://aimilios.bearblog.dev/reverse-engineering-sleep-mask/
4. **IBM tripling entry-level jobs after finding the limits of AI adoption** - https://fortune.com/2026/02/13/tech-giant-ibm-tripling-gen-z-entry-level-hiring-according-to-chro-rewriting-jobs-ai-era/
5. **Ooh.directory: a place to find good blogs that interest you** - https://ooh.directory/
